# Individual Project Planning Stage

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(lubridate)
library(ggplot2)
options(repr.matrix.max.rows = 10)

## Data 

#### players.csv 
This data set contains information about players who used the Minecraft server, PLAICraft. There are 196 observations and 7 variables as follows: 
- `experience` - character: player's experience level (`Beginner`, `Amateur`, `Regular`, `Vetern`, or `Pro`)
- `subscribe` - logical: player's subscription status to a game-related newsletter (`TRUE` or `FALSE`)
- `hashedEmail` - character: hashed player's email
- `played_hours` - double: total hours played by each user 
- `name` - character: player's first name
- `gender` - character: player's gender
- `Age` - double: player's age 
<br>
<br>
Some potential issues with this data are related to the method of data collection since of the variables were collected via self-reporting. This is especially notable with `experience` due to the subjectivity of the category. There are also two missing values in the Age column that should be considered. 

#### sessions.csv
This data set contains information about sessions played on the Minecraft server, PLAICraft. There are 1535 observations and 5 variables as follows:
- `hashedEmail` - character: hashed player's email
- `start_time` - character: session start time in dd/mm/yyyy time 
- `end_time` - character: session end time in dd/mm/yyyy time 
- `orginal_start_time` - double: session start time in milliseconds since January 1st, 1970 at Coordinated Universal Time (UNIX time)
- `orginal_end_time` - double: session end time in milliseconds since January 1st, 1970 at Coordinated Universal Time (UNIX time)
<br>
<br>
Due to multiple observations in the same cell (date and time) for `start_time` and `end_time`, the data is not tidy. Additionally, it should be noted that the server allocates limited PLAItime with dependency on the ability to interact with other players for more time. This could impact session length as players playing alone would not be able to continue playing after 30 minutes. 

In [ ]:
#Loading Data Sets
players_url <- "https://raw.githubusercontent.com/skylv777/Data_Science_Project/refs/heads/main/data/players.csv"
sessions_url <- "https://raw.githubusercontent.com/skylv777/Data_Science_Project/refs/heads/main/data/sessions.csv"
players_data <- read_csv(players_url)
sessions_data <- read_csv(sessions_url)
head(players_data)
head(sessions_data)

In [ ]:
#Tidying sessions_data so there is only one value per cell
sessions_data_tidy <- sessions_data |>
         mutate(start_time = dmy_hm(start_time)) |>
         mutate(end_time = dmy_hm(end_time))
head(sessions_data_tidy)

#### Summary Statistics for players_data

In [ ]:
#Summary Statistics on Quantitative Values of players_data 
players_data_select <- select(players_data, Age, played_hours)

players_data_stats <- 
        bind_rows(map_df(players_data_select, mean, na.rm = TRUE), 
                  map_df(players_data_select, median, na.rm = TRUE),
                  map_df(players_data_select, ~{
    x <- na.omit(.x)
    if (length(x) == 0) return(NA_real_)
    tibble(val = x) |>
      count(val, sort = TRUE) |>
      filter(n == max(n)) |>
      slice_head(n = 1) |>   
      pull(val)}),
                  map_df(players_data_select, min, na.rm = TRUE),
                  map_df(players_data_select, max, na.rm = TRUE), 
                  map_df(players_data_select, sd, na.rm = TRUE), #?seems too high
                  map_df(players_data_select, ~ quantile(.x, probs = 0.25, na.rm = TRUE)[[1]]),
                  map_df(players_data_select, ~ quantile(.x, probs = 0.5, na.rm = TRUE)[[1]]),
                  map_df(players_data_select, ~ quantile(.x, probs = 0.75, na.rm = TRUE)[[1]])) |>
        mutate(Summary = c("Mean", "Median", "Mode", "Minimum", "Maximum", "Standard Deviation", "1st Quartile", "2nd Quartile", "3rd Quartile")) |>
        relocate(Summary) |>
        mutate(across(Age:played_hours, \(x) round(x, digits = 2)))
        
players_data_stats

In [ ]:
player_gender_total <- players_data |> 
        summarize(total = n())|>
        pull()

player_gender_count <- players_data |>
        group_by(gender) |>
        summarize(count=n()) |>
        arrange(desc(count)) |>
        mutate(percent = count / player_gender_total * 100)

player_gender_stats <- player_gender_count |>
        mutate(percent = round(percent, digit = 2))

player_gender_stats

In [ ]:
player_experience_total <- players_data |>
        summarize(total = n()) |>
        pull()

player_experience_count <- players_data |>
        group_by(experience) |>
        summarize(count = n()) |>
        arrange(desc(count)) |>
        mutate(percent = count / player_experience_total * 100)

player_experience_stats <- player_experience_count |>
        mutate(percent = round(percent, digits = 2))
            
player_experience_stats

In [ ]:
player_subscribe_total <- players_data |> 
        summarize(total = n()) |>
        pull()

player_subscribe_count <- players_data |>
        group_by(subscribe) |>
        summarize(count = n()) |>
        arrange(desc(count))|>
        mutate(percent = count / player_subscribe_total * 100)

player_subscribe_stats <- player_subscribe_count |>
        mutate(percent = round(percent, digits = 2))

player_subscribe_stats 

#### Summary Statistics for sessions_data

In [ ]:
sessions_data_difference <- sessions_data_tidy |>
        mutate(session_length = end_time - start_time) |>
        mutate(session_length = as.double(session_length))

sessions_data_select <- select(sessions_data_difference,-hashedEmail)
sessions_data_select_2 <- select(sessions_data_select, original_start_time, original_end_time, session_length)
sessions_data_stats <- 
        bind_rows(map_df(sessions_data_select, mean, na.rm = TRUE), 
                  map_df(sessions_data_select, median, na.rm = TRUE),
                  map_df(sessions_data_select, ~{
    x <- na.omit(.x)
    if (length(x) == 0) return(NA_real_)
    tibble(val = x) |>
      count(val, sort = TRUE) |>
      filter(n == max(n)) |>
      slice_head(n = 1) |>   
      pull(val)}),
                  map_df(sessions_data_select, min, na.rm = TRUE),
                  map_df(sessions_data_select, max, na.rm = TRUE), 
                  map_df(sessions_data_select_2, sd, na.rm = TRUE), 
                  tibble(start_time = quantile(pull(sessions_data_difference, as.double(start_time)), c(0.25, 0.5, 0.75), na.rm = TRUE),
                  end_time = quantile(pull(sessions_data_difference, as.double(end_time)), c(0.25, 0.5, 0.75), na.rm = TRUE), 
                  original_start_time = quantile(pull(sessions_data_difference, original_start_time), c(0.25, 0.5, 0.75), na.rm = TRUE), 
                  original_end_time = quantile(pull(sessions_data_difference, original_end_time), c(0.25, 0.5, 0.75), na.rm = TRUE),
                  session_length = quantile(pull(sessions_data_difference, session_length), c(0.25, 0.5, 0.75), na.rm = TRUE)))|>
       mutate(Summary = c("Mean", "Median", "Mode", "Minimum", "Maximum", "Standard Deviation", "1st Quartile", "2nd Quartile", "3rd Quartile")) |>
       relocate(Summary) |>
       mutate(session_length = round(session_length, digits = 2))
sessions_data_stats

### Visualizations

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 15)
hours_age_subscribe_plot <- players_data |>
        ggplot(aes(x = Age, y = played_hours, colour = subscribe)) +
        geom_point() +
        labs (x = "Age",
              y = "Total hours played",
              colour = "Subscription Status to Game-Related Newsletter") +
        ggtitle("Total hours played versus Age")
hours_age_subscribe_plot      

It is notable that most of the user's have total hours played under 10 hours older than 30 and that the majority of users are under the age of 30. Additionally, all the players who played more than 20 hours total were subscribed to a newsletter and similarily it appears that the users over 30 are considerably more likely to be subscribed to a newsletter. 

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 15)
hours_age_experience_plot <- players_data |>
        ggplot(aes(x = Age, y = played_hours, colour = experience)) +
        geom_point() +
        labs (x = "Age",
              y = "Total hours played",
              colour = "Experience") +
        ggtitle("Total hours played versus Age")
hours_age_experience_plot      

There does not appear to be any clear relationship between total hours played and experience or with age. However, all the users with over approximately 50 total hours played are either amateur or regular. 

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 15)
# Determining average session length per player
average_sessions_data <- sessions_data_tidy |>
         mutate(session_length = end_time - start_time) |>
         mutate(session_length = as.double(session_length)) |>
         group_by(hashedEmail) |>
         select(session_length) |>
         summarize(average_session_length = mean(session_length))

# Combining Data Sets
combined_data <- merge(players_data, average_sessions_data)

average_age_plot <- combined_data |>
        ggplot(aes(x = Age, y = average_session_length, colour = subscribe)) +
        geom_point() +
        labs (x = "Age",
              y = "Average Session Length (Minutes)",
              colour = "Subscription Status to Game-Related Newsletter") +
        ggtitle("Average Session Length versus Age")
average_age_plot  

It appears that the ratio between users subscribed to users not subscribed to a newsletter is higher with players with average session length above 25 minutes. Additionally, users above the age of 30 are more likely to have average session length below 25 minutes in comparison to players under 30. 

In [ ]:
average_age_experience_plot <- combined_data |>
        ggplot(aes(x = Age, y = average_session_length, colour = experience)) +
        geom_point() +
        labs (x = "Age",
              y = "Average Session Length (Minutes)",
              colour = "Experience") +
        ggtitle("Average Session Length versus Age")
average_age_experience_plot  

There does not appear to be any strong relationship between experience and age or average session length. 

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 10)
experience_bar <- players_data |>
        ggplot(aes(x = experience)) +
        geom_bar(aes(fill = subscribe)) +
        ggtitle("Subscription Status by Experience") +
        labs(x = "Experience Level",
             y = "Amount",
             fill = "Subscription Status")
experience_bar

This plot shows the distribution of experience level as well as the ratio of players at each experience level who are subscribed to game-related newsletters. It is notable that there are varying amounts of users of each experience level and that there are significantly more players subscribed to a newsletter than not.

## Question
#### Broad Question: 
Question 1: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?
<br>
<br>
#### Specific Question:
Can hours played, age, and average session length predict whether a user is subscribed to a game-related newsletter? Additionally does this differ by experience?
<br> 
<br>
The data contains the ages, hours played, subscription status, and experience of each user. By combining players.csv and sessions.csv, I can mutate the start and end time of each session to find the length as well as group_by `hashedEmail` to determine the average session length for each player. With this information I will be able to train a classification algorithm to ascertain the answer to my question. 

### Methods and Plan

In order to answer my specific question with the given data, a k-nn classification model would be the most appropriate. Out of the methods introduced in DSCI 100, classification is the only one that predicts a class and therefore more fitting to predict subscription status than regression. 

K-NN classification does not make many assumptions about the distribution of the data however it does assume that data points close to each other belong to the same class. 

K-NN classification is biased with imbalanced data (ie significantly more of one class in the data set) and large numbers of predictors. It is also sensitive to the scaling of the predictors and potential outliers. To combat these weaknesses, it will be important to scale and center the predictors as well as consider balancing the data. 

To build my K-NN classification model, I will first split the data into 80% training and 20% testing. In order to determine which of the variables (hours played, age, and average session length) are optimal to use as predictors, I will create a separate model for each combination of predictors and tune each one using 5-fold cross validation. Then I can pick the combination of predictors that yield the highest cross validation accuracy. Furthermore, I can test the accuracy of my model by utilizing the testing data. 

In order to assess how experience might impact the prediction of subscription status to a game-related newsletter, I can group the training data by experience then train different models that are tuned with 5-fold cross validation. From there, I can compare the cross validation accuracies. 